In [1]:
import os
import sys

import numpy as np

path_to_project = os.path.dirname(os.path.abspath("")) + "/"
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("decision-focused-learning-codebase"))))

In [2]:
from src.utils.experiments import run, update_config
from src.utils.optuna import SearchSpaceConfig, create_study, save_progress

(CVXPY) Aug 21 03:30:32 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.14.6206). Expected < 9.12.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Aug 21 03:30:32 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.14.6206). Expected < 9.12.0. Please open a feature request on cvxpy to enable support for this version.')


Auto-Sklearn cannot be imported.


### Prepare config

We first define a base config for which all parameters remain fixed

In [3]:
base_config = {
    "model": {"name": "cvxpy_knapsack_pyepo", "num_decisions": 10},
    "data": {"name": "knapsack_pyepo"},
    "problem": {"train_ratio": 0.7, "val_ratio": 0.15},
    "decision_maker": {"name": "differentiable"},
    "runner": {"num_epochs": 3, "use_wandb": False, "experiment_name": "test_tuning"},
}

### Define trial

To run hyperparameters tuning with Optuna, we need to define the 'objective' function that quantifies performance of each trial
Essentially, this is the main function, that reads the parameters values from the search space and uses them for evaluation

In [5]:
def _run_trial(trial, seeds: list):
    assert len(seeds) > 0, "Provide at least one seed!"

    # Parse Optuna Trial to get a config of the values
    trial_config = search_space.get_trial_config(trial)

    # For more robust evaluation, we run each hyperparameter configuration for multiple seeds (same across different configurations)
    # Make sure to use different seeds for the experiments after
    per_seeds_results = []
    for seed in seeds:
        # Update config based on trial
        config = update_config(base_config=base_config, updates_config=trial_config)

        # Initialize all objects with the seed
        config["seed"] = seed
        for key in config:
            if isinstance(config[key], dict) and "seed" in config[key]:
                config[key]["seed"] = seed

        results = run(config, optuna_trial=trial)
        per_seeds_results.append(results)

    try:
        return np.mean(per_seeds_results)
    except optuna.TrialPruned:
        raise

###Define search spaces

In [6]:
# To conduct a hyperparameter tuning experiment, we first need to define the search spaces
search_space = SearchSpaceConfig(path_to_project + "examples/hparams_search_spaces/test_search_config.yaml")
print(search_space.config)

{'decision_maker': {'learning_rate': {'type': 'float', 'low': 1e-05, 'high': 0.001, 'log': True}, 'batch_size': {'type': 'int', 'low': 1, 'high': 512, 'log': True}}}


##Create pruner

In [7]:
# Creating a pruner will help pruning trials that are not promising
import optuna

pruner = optuna.pruners.MedianPruner(
    n_startup_trials=10,  # Number of trials to run before pruning
    n_warmup_steps=15,  # Number of epochs to wait before pruning
    interval_steps=1,  # Interval between pruning checks
    n_min_trials=1,  # Minimum trials required for pruning
)

In [8]:
# Specify study name and the folder where the results are stored
# A database.db file will be created in the folder OUTPUT_DIR
# While different problems/methods can be all inside the same database, it can be convenient to separate it by problem or method
STUDY_NAME = "test_study"  # Note that optuna will continue with an existing study if the study already exists
OUTPUT_DIR = "hparam_optimization_results/"  # Folder where the study database is stored (at OUTPUT_DIR/STUDY_NAME)
os.makedirs(f"{OUTPUT_DIR}/{STUDY_NAME}", exist_ok=True)  # Ensure that the folder exists
study = create_study(
    STUDY_NAME,
    storage_url=f"sqlite:///{OUTPUT_DIR}/{STUDY_NAME}/database.db",
    prunner=pruner,
)

Storage url: sqlite:///hparam_optimization_results//test_study/database.db


[I 2025-08-21 15:30:56,381] Using an existing study with name 'test_study' instead of creating a new one.


In [10]:
# We can set up a dashboard that visualizes results when the study is running, opened in the background.
# Make sure to install the package optuna-dashboard.
# Alternatively open the dashboard through the terminal using:
# optuna-dashboard sqlite:///hparam_optimization_results//test_study//database.db --port=8889
import subprocess

port = "8889"
subprocess.Popen(
    [
        "optuna-dashboard",
        "sqlite:///hparam_optimization_results//test_study//database.db",
        "--port",
        port,
    ]
)

print(f"See dashboard here: http://localhost:{port}")

See dashboard here: http://localhost:8889


In [8]:
# Now we run the study
def objective_fn(trial):
    return _run_trial(trial, seeds=list(range(3)))


study.optimize(objective_fn, n_trials=3, timeout=None, catch=(Exception,), show_progress_bar=True)

  0%|          | 0/3 [00:00<?, ?it/s]

Generating data using knapsack_pyepo
Shape of weights: (1, 10), values: (500, 10), features: (500, 5).
Computing optimal decisions for the entire dataset...
Optimal decisions computed and added to dataset.
Computing optimal objectives for the entire dataset...
Optimal objectives computed and added to dataset.
Shuffling indices before splitting...
Dataset split completed: Train=350, Validation=75, Test=75
Problem mode set to: train
Problem mode set to: train
Epoch 0/3: Starting initial validation...
Problem mode set to: validation
Epoch Results:
validation/sym_rel_regret_mean: 0.3428
validation/rel_regret_mean: 0.4783
validation/mse_mean: 24.5923
validation/x_mean: 0.1484
validation/abs_regret_mean: 5.2566
validation/objective_mean: 8.0268
validation/c_mean: 4.4360
Initial best validation metric (abs_regret): 5.2566070556640625
Starting training...
Epoch: 1/3
Problem mode set to: train


Listening on http://127.0.0.1:8889/
Hit Ctrl-C to quit.



Epoch Results:
validation/sym_rel_regret_mean: 0.3428
train/loss_mean: 6.7126
validation/rel_regret_mean: 0.4783
validation/mse_mean: 24.5923
validation/x_mean: 0.1484
train/abs_regret_mean: 6.7126
validation/abs_regret_mean: 5.2566
validation/objective_mean: 8.0268
train/grad_norm_mean: 0.0000
train/solver_calls_mean: 324.5000
validation/c_mean: 4.4360
Problem mode set to: validation
Epoch Results:
validation/sym_rel_regret_mean: 0.3439
train/loss_mean: 6.7126
validation/rel_regret_mean: 0.4793
validation/mse_mean: 24.6950
validation/x_mean: 0.1483
train/abs_regret_mean: 6.7126
validation/abs_regret_mean: 5.2652
validation/objective_mean: 8.0182
train/grad_norm_mean: 0.0000
train/solver_calls_mean: 324.5000
validation/c_mean: 4.4455
Validation evaluation (abs_regret): 5.273838043212891
Epoch: 2/3
Problem mode set to: train
Epoch Results:
validation/sym_rel_regret_mean: 0.3439
train/loss_mean: 6.8400
validation/rel_regret_mean: 0.4793
validation/mse_mean: 24.6950
validation/x_mean: 0.1

/Users/noah/Documents/Python projects/decision-focused-learning-codebase/.venv/lib/python3.9/site-packages/optuna/trial/_trial.py:497: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(


Epoch Results:
validation/sym_rel_regret_mean: 0.2801
train/loss_mean: 5.4740
validation/rel_regret_mean: 0.4062
validation/mse_mean: 51.9513
validation/x_mean: 0.1460
train/abs_regret_mean: 5.4740
validation/abs_regret_mean: 4.6309
validation/objective_mean: 8.8030
train/grad_norm_mean: 0.0000
train/solver_calls_mean: 537.0000
validation/c_mean: 4.2324
Problem mode set to: validation
Epoch Results:
validation/sym_rel_regret_mean: 0.2860
train/loss_mean: 5.4740
validation/rel_regret_mean: 0.4125
validation/mse_mean: 52.0142
validation/x_mean: 0.1473
train/abs_regret_mean: 5.4740
validation/abs_regret_mean: 4.7370
validation/objective_mean: 8.6969
train/grad_norm_mean: 0.0000
train/solver_calls_mean: 537.0000
validation/c_mean: 4.2235
Validation evaluation (abs_regret): 4.949247360229492
Epoch: 3/3
Problem mode set to: train


/Users/noah/Documents/Python projects/decision-focused-learning-codebase/.venv/lib/python3.9/site-packages/optuna/trial/_trial.py:497: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(


Epoch Results:
validation/sym_rel_regret_mean: 0.2860
train/loss_mean: 5.4677
validation/rel_regret_mean: 0.4125
validation/mse_mean: 52.0142
validation/x_mean: 0.1473
train/abs_regret_mean: 5.4677
validation/abs_regret_mean: 4.7370
validation/objective_mean: 8.6969
train/grad_norm_mean: 0.0000
train/solver_calls_mean: 749.5000
validation/c_mean: 4.2235
Problem mode set to: validation
Epoch Results:
validation/sym_rel_regret_mean: 0.2909
train/loss_mean: 5.4677
validation/rel_regret_mean: 0.4182
validation/mse_mean: 52.0539
validation/x_mean: 0.1480
train/abs_regret_mean: 5.4677
validation/abs_regret_mean: 5.0010
validation/objective_mean: 8.4328
train/grad_norm_mean: 0.0000
train/solver_calls_mean: 749.5000
validation/c_mean: 4.2191
Validation evaluation (abs_regret): 5.793196678161621
Training finished. Evaluating on the test set...
Problem mode set to: test
Epoch Results:
validation/sym_rel_regret_mean: 0.2909
test/c_mean: 4.2413
train/loss_mean: 5.4677
test/mse_mean: 69.7304
valida

/Users/noah/Documents/Python projects/decision-focused-learning-codebase/.venv/lib/python3.9/site-packages/optuna/trial/_trial.py:497: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(


Optimal decisions computed and added to dataset.
Computing optimal objectives for the entire dataset...
Optimal objectives computed and added to dataset.
Shuffling indices before splitting...
Dataset split completed: Train=350, Validation=75, Test=75
Problem mode set to: train
Problem mode set to: train
Epoch 0/3: Starting initial validation...
Problem mode set to: validation
Epoch Results:
validation/sym_rel_regret_mean: 0.2612
validation/rel_regret_mean: 0.3708
validation/mse_mean: 57.6359
validation/x_mean: 0.1763
validation/abs_regret_mean: 8.5667
validation/objective_mean: 9.5209
validation/c_mean: 3.8630
Initial best validation metric (abs_regret): 8.566689491271973
Starting training...
Epoch: 1/3
Problem mode set to: train
Epoch Results:
validation/sym_rel_regret_mean: 0.2612
train/loss_mean: 6.8853
validation/rel_regret_mean: 0.3708
validation/mse_mean: 57.6359
validation/x_mean: 0.1763
train/abs_regret_mean: 6.8853
validation/abs_regret_mean: 8.5667
validation/objective_mean: 

In [9]:
# The dashboard summarizes the results. Alternatively results can be retrieved as follows.
import optuna

path = f"sqlite:///{OUTPUT_DIR}/{STUDY_NAME}/database.db"
studies = optuna.study.get_all_study_summaries(storage=path)

for study_summary in studies:
    study_name = study_summary.study_name
    study = optuna.load_study(study_name=study_name, storage=path)

    print(
        f"Results study: {study_name}\n"
        f"Completed trials: {len(study.trials)}\n"
        f"Best value: {study.best_value:.4f}\n"
        f"Best parameters: {study.best_params}"
    )

Results study: test_study
Completed trials: 8
Best value: 4.9115
Best parameters: {'learning_rate': 0.0006497345283948292, 'batch_size': 215}


In [10]:
# We save the best configuration to a YAML file
save_progress(study, search_space, OUTPUT_DIR)

Best configuration saved to hparam_optimization_results/best_config.yaml
